In [1]:
# Spotify API
pip install pandas scikit-learn spotipy


  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Uninstalling urllib3-1.25.8:
      Successfully uninstalled urllib3-1.25.8
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
ERROR: astroid 2.3.3 requires typed-ast<1.5,>=1.4.0; implementation_name == "cpython" and python_version < "3.8", which is not installed.


In [58]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Set up authentication
client_id = '3f7574b3a297422986c8bc4d831550fa'
client_secret = '66bffc847689407bbddce02c9db0b152'
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

# Get playlist tracks
playlist_id = 'https://open.spotify.com/playlist/6VS2rEneGDah7Lya2E5mCR?si=bd26bd0518ad43c9'
results = sp.playlist_tracks(playlist_id)
tracks = results['items']
track_names = []

# Storing audio features of all tracks
audio_data = []
for track in tracks:
    track_names.append(track['track']['name'])
    audio_data += sp.audio_features(track['track']['uri'])
    
audio_df = pd.DataFrame(audio_data)
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
            'loudness', 'speechiness', 'tempo', 'valence']
feature_matrix = audio_df[features].values

In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Normalize features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(feature_matrix)

# Cosine Similarity Matrix
similarity_matrix = cosine_similarity(normalized_features)


In [66]:
# Recommend Songs

def recommend_song(song, songs, similarity_matrix, top_n=5):
    # Find the index of the given song ID
    song_idx = songs.index(song)
    
    # Get the similarity scores for this song
    similarity_scores = list(enumerate(similarity_matrix[song_idx]))
    
    # Sort songs by similarity score (excluding the song itself)
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:]
    
    # Get the top N recommendations
    top_recommendations = sorted_scores[:top_n]
    
    # Fetch the recommended song details
    recommendations = []
    for idx, score in top_recommendations:
        recommendations.append({
            'name': songs[idx],
            'similarity_score': score
        })
    return recommendations

recommendations = recommend_song('Sundress', track_names, similarity_matrix, 3)
print(recommendations)    

[{'name': 'LOVE. FEAT. ZACARI.', 'similarity_score': 0.874603028527618}, {'name': 'Locked out of Heaven', 'similarity_score': 0.8318851448692177}, {'name': 'Treasure', 'similarity_score': 0.8291542350124372}]
